<a href="https://colab.research.google.com/github/wooookim/ML-DL/blob/main/%EC%84%9C%EC%9A%B8%EC%8B%9C_%ED%8F%89%EA%B7%A0_%EA%B8%B0%EC%98%A8_%EC%98%88%EC%B8%A1_11(%EC%97%AC%EA%B8%B0%EB%B6%80%ED%84%B0%20%EC%A0%90%EC%88%98%20%EC%A2%8B%EC%95%84%EC%A7%90).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving sample_submission.csv to sample_submission.csv


In [3]:
import pandas as pd
data = pd.read_csv('/content/train.csv')
submission = pd.read_csv('/content/sample_submission.csv')

# '일시' 열을 날짜 데이터로 변환
data['일시'] = pd.to_datetime(data['일시'])

# 계절 분류 함수 정의
def classify_season(month):
    if  month == 1 or month == 2 or month == 12:
        return '겨울'
    elif 3 <= month <= 5:
        return '봄'
    elif 6 <= month <= 8:
        return '여름'
    else:
        return '가을'

# '일시' 컬럼에서 월 정보를 추출하여 '계절' 열 추가
data['월'] = data['일시'].dt.month
data['계절'] = data['월'].apply(classify_season)

# 계절별 중앙값 계산
seasonal_medians = data.groupby('계절').transform('median')

# 결측값 대체
data.fillna(seasonal_medians, inplace=True)

# print(seasonal_medians)

       최고기온  최저기온  일교차  강수량  평균습도  평균풍속  일조합  일사합   일조율  평균기온    월
0       3.4  -4.6  7.8  0.3  60.4   2.2  6.4  8.0  63.7  -0.8  2.0
1       3.4  -4.6  7.8  0.3  60.4   2.2  6.4  8.0  63.7  -0.8  2.0
2       3.4  -4.6  7.8  0.3  60.4   2.2  6.4  8.0  63.7  -0.8  2.0
3       3.4  -4.6  7.8  0.3  60.4   2.2  6.4  8.0  63.7  -0.8  2.0
4       3.4  -4.6  7.8  0.3  60.4   2.2  6.4  8.0  63.7  -0.8  2.0
...     ...   ...  ...  ...   ...   ...  ...  ...   ...   ...  ...
23006   3.4  -4.6  7.8  0.3  60.4   2.2  6.4  8.0  63.7  -0.8  2.0
23007   3.4  -4.6  7.8  0.3  60.4   2.2  6.4  8.0  63.7  -0.8  2.0
23008   3.4  -4.6  7.8  0.3  60.4   2.2  6.4  8.0  63.7  -0.8  2.0
23009   3.4  -4.6  7.8  0.3  60.4   2.2  6.4  8.0  63.7  -0.8  2.0
23010   3.4  -4.6  7.8  0.3  60.4   2.2  6.4  8.0  63.7  -0.8  2.0

[23011 rows x 11 columns]


<ipython-input-3-6cb540a28dc0>:24: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seasonal_medians = data.groupby('계절').transform('median')


**훈련 테스트 분리를 안하고 하면 어떻게 되나**

In [5]:
train = data[['최고기온', '최저기온', '일교차', '강수량', '평균습도', '평균풍속', '일조합', '일사합', '일조율']]
# 일시를 제외해보기로 함
target = data['평균기온']

In [ ]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
xgb.fit(train, target)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor()
rf.fit(train, target)

pred_xgb = xgb.predict(train)
pred_rf = rf.predict(train)

from sklearn.metrics import mean_absolute_error

train_mae_xgb = mean_absolute_error(target, pred_xgb)
test_mae_xgb = mean_absolute_error(target, pred_xgb)

train_mae_rf = mean_absolute_error(target, pred_rf)
test_mae_rf = mean_absolute_error(target, pred_rf)

In [7]:
print('xgb train:', train_mae_xgb)
print('xgb test:', test_mae_xgb)

submission['평균기온'] = pred_xgb[:358]
print(submission.head(10))

submission.to_csv("DEAGYUL_8_submission.csv", index=False)

print(submission.head())

xgb train: 0.28782037605745353
xgb test: 0.28782037605745353
           일시      평균기온
0  2023-01-01 -2.112965
1  2023-01-02 -2.021526
2  2023-01-03  3.954774
3  2023-01-04  6.891986
4  2023-01-05 -4.174560
5  2023-01-06 -5.511215
6  2023-01-07  1.097994
7  2023-01-08  1.897515
8  2023-01-09  2.460873
9  2023-01-10  1.902011
           일시      평균기온
0  2023-01-01 -2.112965
1  2023-01-02 -2.021526
2  2023-01-03  3.954774
3  2023-01-04  6.891986
4  2023-01-05 -4.174560


In [8]:
print('rf train:', train_mae_rf)
print('rf test:', test_mae_rf)

submission['평균기온'] = pred_rf[:358]
print(submission.head(10))

submission.to_csv("DEAGYUL_9_submission.csv", index=False)

print(submission.head())

rf train: 0.14153922037286518
rf test: 0.14153922037286518
           일시   평균기온
0  2023-01-01 -1.641
1  2023-01-02 -2.065
2  2023-01-03  3.723
3  2023-01-04  7.000
4  2023-01-05 -4.129
5  2023-01-06 -5.382
6  2023-01-07  1.225
7  2023-01-08  1.829
8  2023-01-09  2.267
9  2023-01-10  2.013
           일시   평균기온
0  2023-01-01 -1.641
1  2023-01-02 -2.065
2  2023-01-03  3.723
3  2023-01-04  7.000
4  2023-01-05 -4.129


1. 테스트 세트 점수가 계속 에러가 나서 빼고 출력했는데 약간 성능이 높아졌다

2. 어차피 테스트 세트 오류가 나는거 훈련에 넣고 돌리면 더 좋은 예측을 할거라고 생각하고 분리하지 않고 시도해봤다

3. 작성 중간에 훈련/테스트를 분리하면서 계절성을 무시한 분리가 된것 아닌가 하는 생각을 하게 됐다.

4. 훈련-테스트를 어떤 계절이 속한 지점에서 분리해버려서 이렇게 되버렸다고 추측

5. 분리하지 않고 작성해보니 점수가 아주 많이 올랐다

6. 일시에 대한 처리가 어려워서 아예 배제하고 계절로 묶고, 계절의 중앙값(평균은 0이 나오기 때문)으로 대체하고 진행했다.

7. 통상적인 계절로 분류했는데 이를 조정해보면 약간 더 좋아질지도?